# HyperTuning with KerasTuner and TensorFlow
---

Building machine learning models is an iterative process that involves optimizing the model's performance and compute resources. The settings that you adjust during each iteration are called *hyperparameters*. They govern the training process and are held constant during training. 

The process of searching for optimal hyperparameters is called *hyperparameter tuning* or *hypertuning*, and is essential in any machine learning project. Hypertuning helps boost performance and reduces model complexity by removing unnecessary parameters (e.g., number of units in a dense layer).
There are two type of hyperparameters:
1. *Model hyperparameters* that influence model architecture (e.g., number and width of hidden layers in a DNN)
2. *Algorithm hyperparameters* that influence the speed and quality of training (e.g., learning rate and activation function).

The number of hyperparameter combinations, even in a shallow DNN, can grow insanely large making manually searching for the optimal set simply not feasible nor scalable. 
This post will introduce you to KerasTuner, a library made to automate the hyperparameter search. We'll build a deep learning model and train it on the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) with:
* Pre-selected hyperparameters
* Optimized hyperparameters with KerasTuner
* Optimized pre-trained Xception and ResNet models

Let's begin!

## Imports and Preprocessing

In [1]:
import tensorflow as tf
import kerastuner as kt

from tensorflow import keras

print(f"TensorFlow Version: {tf.__version__}")
print(f"KerasTuner Version: {kt.__version__}")

TensorFlow Version: 2.5.0
KerasTuner Version: 1.0.1


In [2]:
# Load and split data into train and test sets
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
# Normalize pixels to values between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

## Baseline Performance
Baseline performance will be judged by training a neural network with pre-selected hyperparameters:
* `1` hidden layer with `512` neurons
* `Adam` optimizer with learning rate of `0.001`
* Dropout layer of `0.2`

In [4]:
# Build baseline model with Sequential API
b_model = keras.Sequential()
b_model.add(keras.layers.Flatten(input_shape=(28,28)))
b_model.add(keras.layers.Dense(units=512, activation='relu', name='dense_1'))
b_model.add(keras.layers.Dropout(0.2))
b_model.add(keras.layers.Dense(10, activation='softmax'))

# Print model summary
b_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


2021-08-05 11:48:33.406905: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Notice how we hardcoded each hyperparameter.  These include the number and width of hidden layers, activation function, and dropout.  

We will now set the optimizer, learning rate, and loss function.

In [5]:
# Set training parameters
b_model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

/opt/anaconda3/envs/hype/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


With our model's setting defined, we are ready to train!

In [6]:
# Number of epochs
NUM_EPOCHS = 20

# Early stopping set after 5 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Train model
b_model.fit(X_train, y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)

Epoch 1/20


2021-08-05 11:48:34.208833: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


1500/1500 - 5s - loss: 0.5145 - accuracy: 0.8180 - val_loss: 0.4199 - val_accuracy: 0.8428
Epoch 2/20
1500/1500 - 6s - loss: 0.3944 - accuracy: 0.8577 - val_loss: 0.4413 - val_accuracy: 0.8307
Epoch 3/20
1500/1500 - 4s - loss: 0.3585 - accuracy: 0.8678 - val_loss: 0.3530 - val_accuracy: 0.8702
Epoch 4/20
1500/1500 - 4s - loss: 0.3331 - accuracy: 0.8767 - val_loss: 0.3300 - val_accuracy: 0.8825
Epoch 5/20
1500/1500 - 4s - loss: 0.3164 - accuracy: 0.8825 - val_loss: 0.3433 - val_accuracy: 0.8756
Epoch 6/20
1500/1500 - 5s - loss: 0.3042 - accuracy: 0.8863 - val_loss: 0.3175 - val_accuracy: 0.8855
Epoch 7/20
1500/1500 - 4s - loss: 0.2920 - accuracy: 0.8889 - val_loss: 0.3280 - val_accuracy: 0.8831
Epoch 8/20
1500/1500 - 5s - loss: 0.2793 - accuracy: 0.8955 - val_loss: 0.3179 - val_accuracy: 0.8881
Epoch 9/20
1500/1500 - 4s - loss: 0.2715 - accuracy: 0.8980 - val_loss: 0.3339 - val_accuracy: 0.8835
Epoch 10/20
1500/1500 - 4s - loss: 0.2618 - accuracy: 0.9017 - val_loss: 0.3393 - val_accurac

We'll create a helper function to evaluate our model and view the results in a dataframe helping us easily compare models later on.

In [7]:
import pandas as pd

def evaluate_model(model, X_test, y_test):
    """
    evaluate model on test set and show results in dataframe.
    
    Parameters
    ----------
    model : keras model
        trained keras model.
    X_test : numpy array
        Features of holdout set.
    y_test : numpy array
        Labels of holdout set.
        
    Returns
    -------
    display_df : DataFrame
        Pandas dataframe containing evaluation results.
    """
    eval_dict = model.evaluate(X_test, y_test, return_dict=True)
    
    display_df = pd.DataFrame([eval_dict.values()], columns=[list(eval_dict.keys())])
    
    return display_df

In [8]:
results = evaluate_model(b_model, X_test, y_test)

results.index = ['Baseline']

results.head()

313/313 [==============================] - ETA: 26s - loss: 0.4353 - accuracy: 0.843 - ETA: 0s - loss: 0.3339 - accuracy: 0.879 - ETA: 0s - loss: 0.3168 - accuracy: 0.88 - ETA: 0s - loss: 0.3434 - accuracy: 0.87 - ETA: 0s - loss: 0.3477 - accuracy: 0.87 - ETA: 0s - loss: 0.3579 - accuracy: 0.87 - ETA: 0s - loss: 0.3535 - accuracy: 0.87 - ETA: 0s - loss: 0.3430 - accuracy: 0.88 - ETA: 0s - loss: 0.3418 - accuracy: 0.88 - 0s 1ms/step - loss: 0.3419 - accuracy: 0.8809


,loss,accuracy
Baseline,0.341941,0.8809


There's the results for a single set of hyperparameters.  Imagine trying out different learning rates, dropout percentages, number of hidden layers, and number of neurons in each hidden layer.  As you can see, manual hypertuning is simply not feasible nor scalable.  In the next section you'll see how KerasTuner solves these problems simply by automating the process and searching the hyperparameter space in an efficient way.  

## Keras Tuner

In [17]:
def build_model(hp):
    """
    Builds model and sets up hyperparameter space to search.
    
    Parameters
    ----------
    hp : keras tuner object
        Used to sample hyperparameters.
        
    Returns
    -------
    model : keras model
        Model with hyperparameters to tune.
    """
    # Initialize sequential API and start building model.
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    
    # Tune the number of layers and units in each hidden layer.
    # Number of layers: 1 - 5
    # Number of Units: 32 - 512 with stepsize of 32
    for i in range(1, hp.Int("num_layers", 2, 6)):
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                activation="relu")
            )
        
        # Tune dropout layer with values from 0 - 0.3.
        model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    
    # Add output layer.
    model.add(keras.layers.Dense(units=10, activation="softmax"))
    
    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=["accuracy"])
    
    return model

In [18]:
# Instantiate the tuner
tuner = kt.Hyperband(build_model,
                     objective="val_accuracy",
                     max_epochs=20,
                     factor=3,
                     hyperband_iterations=10,
                     directory="kt_dir",
                     project_name="kt_hyperband",
                     overwrite="True")

In [19]:
# Display search space summary
tuner.search_space_summary()

In [20]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train, y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)

Epoch 1/3
1500/1500 - 3s - loss: 0.8351 - accuracy: 0.7197 - val_loss: 0.5332 - val_accuracy: 0.8166
Epoch 2/3
1500/1500 - 3s - loss: 0.4929 - accuracy: 0.8274 - val_loss: 0.4468 - val_accuracy: 0.8440
Epoch 3/3
1500/1500 - 3s - loss: 0.4337 - accuracy: 0.8461 - val_loss: 0.4031 - val_accuracy: 0.8577


Epoch 1/3
1500/1500 - 5s - loss: 0.7408 - accuracy: 0.7332 - val_loss: 0.4649 - val_accuracy: 0.8361
Epoch 2/3
1500/1500 - 5s - loss: 0.4695 - accuracy: 0.8327 - val_loss: 0.4117 - val_accuracy: 0.8539
Epoch 3/3
1500/1500 - 5s - loss: 0.4185 - accuracy: 0.8492 - val_loss: 0.3764 - val_accuracy: 0.8635


Epoch 1/3
1500/1500 - 3s - loss: 0.8354 - accuracy: 0.7080 - val_loss: 0.4946 - val_accuracy: 0.8238
Epoch 2/3
1500/1500 - 2s - loss: 0.5146 - accuracy: 0.8178 - val_loss: 0.4266 - val_accuracy: 0.8473
Epoch 3/3
1500/1500 - 3s - loss: 0.4441 - accuracy: 0.8432 - val_loss: 0.3966 - val_accuracy: 0.8560


Epoch 1/3
1500/1500 - 7s - loss: 0.8235 - accuracy: 0.7068 - val_loss: 0.6554 - val_accuracy: 0.7373
Epoch 2/3
1500/1500 - 6s - loss: 0.7662 - accuracy: 0.7306 - val_loss: 0.6312 - val_accuracy: 0.7820
Epoch 3/3
1500/1500 - 6s - loss: 0.7634 - accuracy: 0.7314 - val_loss: 0.5826 - val_accuracy: 0.8102


Epoch 1/3
1500/1500 - 5s - loss: 0.8389 - accuracy: 0.6942 - val_loss: 0.5197 - val_accuracy: 0.8139
Epoch 2/3
1500/1500 - 5s - loss: 0.5299 - accuracy: 0.8096 - val_loss: 0.4307 - val_accuracy: 0.8464
Epoch 3/3
1500/1500 - 5s - loss: 0.4654 - accuracy: 0.8329 - val_loss: 0.4065 - val_accuracy: 0.8491


...

Epoch 1/20
1500/1500 - 3s - loss: 0.8310 - accuracy: 0.6999 - val_loss: 0.6250 - val_accuracy: 0.7873
Epoch 2/20
1500/1500 - 2s - loss: 0.7220 - accuracy: 0.7391 - val_loss: 0.5993 - val_accuracy: 0.8013
Epoch 3/20
1500/1500 - 2s - loss: 0.7031 - accuracy: 0.7455 - val_loss: 0.5794 - val_accuracy: 0.7977
Epoch 4/20
1500/1500 - 3s - loss: 0.7000 - accuracy: 0.7492 - val_loss: 0.5767 - val_accuracy: 0.8053
Epoch 5/20
1500/1500 - 3s - loss: 0.6943 - accuracy: 0.7504 - val_loss: 0.6076 - val_accuracy: 0.7740
Epoch 6/20
1500/1500 - 2s - loss: 0.6897 - accuracy: 0.7558 - val_loss: 0.5997 - val_accuracy: 0.7840
Epoch 7/20
1500/1500 - 2s - loss: 0.6818 - accuracy: 0.7560 - val_loss: 0.5493 - val_accuracy: 0.8163
Epoch 8/20
1500/1500 - 2s - loss: 0.6692 - accuracy: 0.7609 - val_loss: 0.6149 - val_accuracy: 0.7832
Epoch 9/20
1500/1500 - 2s - loss: 0.6622 - accuracy: 0.7626 - val_loss: 0.5443 - val_accuracy: 0.8219
Epoch 10/20
1500/1500 - 2s - loss: 0.6727 - accuracy: 0.7591 - val_loss: 0.5609 - 

Epoch 1/20
1500/1500 - 5s - loss: 0.6457 - accuracy: 0.7794 - val_loss: 0.4887 - val_accuracy: 0.8328
Epoch 2/20
1500/1500 - 5s - loss: 0.5473 - accuracy: 0.8141 - val_loss: 0.4713 - val_accuracy: 0.8406
Epoch 3/20
1500/1500 - 5s - loss: 0.5233 - accuracy: 0.8215 - val_loss: 0.4922 - val_accuracy: 0.8382
Epoch 4/20
1500/1500 - 5s - loss: 0.5266 - accuracy: 0.8274 - val_loss: 0.4937 - val_accuracy: 0.8232
Epoch 5/20
1500/1500 - 4s - loss: 0.5151 - accuracy: 0.8279 - val_loss: 0.5328 - val_accuracy: 0.8287
Epoch 6/20
1500/1500 - 4s - loss: 0.5044 - accuracy: 0.8295 - val_loss: 0.4943 - val_accuracy: 0.8416
Epoch 7/20
1500/1500 - 4s - loss: 0.5076 - accuracy: 0.8325 - val_loss: 0.5102 - val_accuracy: 0.8251


Epoch 1/20
1500/1500 - 2s - loss: 0.5991 - accuracy: 0.7878 - val_loss: 0.4182 - val_accuracy: 0.8442
Epoch 2/20
1500/1500 - 2s - loss: 0.4398 - accuracy: 0.8420 - val_loss: 0.3853 - val_accuracy: 0.8577
Epoch 3/20
1500/1500 - 2s - loss: 0.3954 - accuracy: 0.8568 - val_loss: 0.3611 - val_accuracy: 0.8674
Epoch 4/20
1500/1500 - 2s - loss: 0.3696 - accuracy: 0.8669 - val_loss: 0.3580 - val_accuracy: 0.8710
Epoch 5/20
1500/1500 - 2s - loss: 0.3523 - accuracy: 0.8721 - val_loss: 0.3355 - val_accuracy: 0.8763
Epoch 6/20
1500/1500 - 2s - loss: 0.3373 - accuracy: 0.8765 - val_loss: 0.3602 - val_accuracy: 0.8717
Epoch 7/20
1500/1500 - 2s - loss: 0.3211 - accuracy: 0.8814 - val_loss: 0.3386 - val_accuracy: 0.8817
Epoch 8/20
1500/1500 - 2s - loss: 0.3120 - accuracy: 0.8866 - val_loss: 0.3442 - val_accuracy: 0.8778
Epoch 9/20
1500/1500 - 2s - loss: 0.3036 - accuracy: 0.8879 - val_loss: 0.3244 - val_accuracy: 0.8840
Epoch 10/20
1500/1500 - 2s - loss: 0.2925 - accuracy: 0.8904 - val_loss: 0.3353 - 

Epoch 1/20
1500/1500 - 3s - loss: 0.7214 - accuracy: 0.7597 - val_loss: 0.4909 - val_accuracy: 0.8288
Epoch 2/20
1500/1500 - 3s - loss: 0.4700 - accuracy: 0.8342 - val_loss: 0.4355 - val_accuracy: 0.8468
Epoch 3/20
1500/1500 - 2s - loss: 0.4154 - accuracy: 0.8529 - val_loss: 0.3951 - val_accuracy: 0.8612
Epoch 4/20
1500/1500 - 2s - loss: 0.3852 - accuracy: 0.8622 - val_loss: 0.3740 - val_accuracy: 0.8655
Epoch 5/20
1500/1500 - 3s - loss: 0.3625 - accuracy: 0.8699 - val_loss: 0.3556 - val_accuracy: 0.8748
Epoch 6/20
1500/1500 - 3s - loss: 0.3448 - accuracy: 0.8749 - val_loss: 0.3497 - val_accuracy: 0.8743
Epoch 7/20
1500/1500 - 3s - loss: 0.3301 - accuracy: 0.8804 - val_loss: 0.3367 - val_accuracy: 0.8808
Epoch 8/20
1500/1500 - 3s - loss: 0.3196 - accuracy: 0.8846 - val_loss: 0.3307 - val_accuracy: 0.8812
Epoch 9/20
1500/1500 - 3s - loss: 0.3087 - accuracy: 0.8867 - val_loss: 0.3274 - val_accuracy: 0.8829
Epoch 10/20
1500/1500 - 3s - loss: 0.3008 - accuracy: 0.8905 - val_loss: 0.3342 - 

INFO:tensorflow:Oracle triggered exit


In [21]:
# Get the optimal hyperparameters from the results
best_hps=tuner.get_best_hyperparameters()[0]

In [22]:
h_model = tuner.hypermodel.build(best_hps)
h_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 288)               226080    
_________________________________________________________________
dropout (Dropout)            (None, 288)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               147968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0

In [23]:
# Train the hypertuned model
h_model.fit(X_train, y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)

Epoch 1/20
1500/1500 - 10s - loss: 0.7174 - accuracy: 0.7520 - val_loss: 0.4340 - val_accuracy: 0.8469
Epoch 2/20
1500/1500 - 10s - loss: 0.4468 - accuracy: 0.8451 - val_loss: 0.3856 - val_accuracy: 0.8636
Epoch 3/20
1500/1500 - 8s - loss: 0.3888 - accuracy: 0.8619 - val_loss: 0.3677 - val_accuracy: 0.8648
Epoch 4/20
1500/1500 - 10s - loss: 0.3574 - accuracy: 0.8736 - val_loss: 0.3366 - val_accuracy: 0.8797
Epoch 5/20
1500/1500 - 8s - loss: 0.3327 - accuracy: 0.8811 - val_loss: 0.3358 - val_accuracy: 0.8783
Epoch 6/20
1500/1500 - 9s - loss: 0.3120 - accuracy: 0.8876 - val_loss: 0.3391 - val_accuracy: 0.8774
Epoch 7/20
1500/1500 - 8s - loss: 0.2948 - accuracy: 0.8921 - val_loss: 0.3651 - val_accuracy: 0.8648
Epoch 8/20
1500/1500 - 10s - loss: 0.2801 - accuracy: 0.8984 - val_loss: 0.3138 - val_accuracy: 0.8873
Epoch 9/20
1500/1500 - 8s - loss: 0.2667 - accuracy: 0.9028 - val_loss: 0.3175 - val_accuracy: 0.8898
Epoch 10/20
1500/1500 - 8s - loss: 0.2585 - accuracy: 0.9048 - val_loss: 0.321

In [24]:
hyper_df = evaluate_model(h_model, X_test, y_test)

hyper_df.index = ["Hypertuned"]

results.append(hyper_df)

313/313 [==============================] - ETA: 32s - loss: 0.6189 - accuracy: 0.906 - ETA: 0s - loss: 0.3634 - accuracy: 0.889 - ETA: 0s - loss: 0.3701 - accuracy: 0.88 - ETA: 0s - loss: 0.3467 - accuracy: 0.89 - ETA: 0s - loss: 0.3459 - accuracy: 0.89 - ETA: 0s - loss: 0.3799 - accuracy: 0.88 - ETA: 0s - loss: 0.3767 - accuracy: 0.88 - ETA: 0s - loss: 0.3840 - accuracy: 0.88 - ETA: 0s - loss: 0.3928 - accuracy: 0.88 - ETA: 0s - loss: 0.3868 - accuracy: 0.88 - ETA: 0s - loss: 0.3768 - accuracy: 0.88 - ETA: 0s - loss: 0.3682 - accuracy: 0.88 - ETA: 0s - loss: 0.3710 - accuracy: 0.88 - 1s 2ms/step - loss: 0.3656 - accuracy: 0.8892


,loss,accuracy
Baseline,0.341941,0.8809
Hypertuned,0.365617,0.8892
